In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
import tensorflow as tf
#import tensorflow_addons as tfa

In [10]:
img_size = (224, 224)

# train data
cwd = os.getcwd()
epochs = 100
df = pd.read_csv(cwd + '/state/train.csv')
X = list(df['image'][i] for i in range(len(df)))
y = list(df['emotion'][i] for i in range(len(df)))

In [11]:
weight = dict(enumerate(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)))
print(weight)

{0: 1.7054881133029842, 1: 1.8921156004489337, 2: 1.7273309426229508, 3: 1.595716990061524, 4: 0.4823676680972818, 5: 0.4303995404646413, 6: 1.3411893396976928, 7: 1.8607891832229582}


In [12]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(img_size[0], img_size[1], 3)),
        tf.keras.layers.experimental.preprocessing.RandomRotation(
            0.05, fill_mode='nearest'
        ),

        tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
            include_top=False, weights='imagenet', input_tensor=None,
            input_shape=(img_size[0], img_size[1], 3), include_preprocessing=True
        ),


        # tf.keras.layers.GlobalAveragePooling2D(),
        # tf.keras.layers.BatchNormalization(),
        # tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Flatten(),


        tf.keras.layers.Dense(512, activation='relu'),
        # tf.keras.layers.BatchNormalization(),
        # tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(256, activation='relu'),
        # tf.keras.layers.BatchNormalization(),
        # tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(128, activation='relu'),
        # tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(8, activation='softmax')

    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(amsgrad=True),  # tf.keras.optimizers.Adam(amsgrad=True),  # tf.keras.optimizers.SGD(nesterov=True), # tfa.optimizers.RectifiedAdam(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(
                      from_logits=True),
                  metrics=['accuracy'])

    return model

In [13]:
# es_callback = tf.keras.callbacks.EarlyStopping(
#     monitor='val_accuracy', patience=epochs*0.05, restore_best_weights=True)
# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath="model/cp-{epoch:04d}.ckpt",
#     verbose=0,
#     save_weights_only=True,
#     monitor='val_accuracy',
#     save_best_only=True)
# tb_callback = tf.keras.callbacks.TensorBoard(
#     log_dir="./logs",
#             update_freq="epoch")

In [14]:
model = create_model()
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_rotation_1 (RandomRo  (None, 224, 224, 3)      0         
 tation)                                                         
                                                                 
 efficientnetv2-b0 (Function  (None, 7, 7, 1280)       5919312   
 al)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 62720)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               32113152  
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 128)              

In [15]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
# adam=Adam(lr=0.001)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])


In [16]:
import os, cv2, shutil
df_train = pd.read_csv(cwd + "\\state\\train.csv")
df_test = pd.read_csv(cwd + "\\state\\test.csv")
output_train_dir = cwd + "\\train"
root_image_dir = cwd + "\\..\\cleaned_images\\"
dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
dir_test = cwd + "\\test\\"
dir_train = cwd + "\\train\\"
if(os.path.exists(dir_train) or os.path.exists(dir_test)):
    shutil.rmtree(dir_train)
    shutil.rmtree(dir_test)
for emotion in dictionary:
    os.makedirs(dir_test + "\\" + emotion)
    os.makedirs(dir_train + "\\" + emotion)

def isInDir(image_dir):
    return len(image_dir.split("/")) >= 2
for index, row in df_test.iterrows():
    if(isInDir(row['image']) == False): continue
    output_test_dir = dir_test + row['image']
    image = cv2.imread(root_image_dir + row['image'])
    cv2.imwrite(output_test_dir, image)
for index, row in df_train.iterrows():
    if(isInDir(row['image']) == False): continue
    output_train_dir = dir_train + row['image']
    image = cv2.imread(root_image_dir + row['image'])
    cv2.imwrite(output_train_dir, image)


In [17]:
bs = 30
train_dir = cwd + "\\train"
test_dir = cwd + "\\test"
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(img_size[0],img_size[1]))
validation_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(img_size[0],img_size[1]))
dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    class_names=dictionary,
    color_mode='rgb',
    batch_size=bs)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    class_names=dictionary,
    color_mode='rgb',
    batch_size=bs)

Found 19175 images belonging to 8 classes.
Found 2367 images belonging to 8 classes.
Found 19175 files belonging to 8 classes.
Found 2367 files belonging to 8 classes.


In [18]:

es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=epochs*0.05, restore_best_weights=True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model/cp-{epoch:04d}.ckpt",
    verbose=0,
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=True)
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir="./logs",
            update_freq="epoch")

In [20]:
history = model.fit(train_ds, validation_data=val_ds, shuffle=True, epochs=epochs, callbacks=[es_callback, cp_callback, tb_callback
                                                                                                  # , reduce_lr
                                                                                                  ], class_weight=weight, batch_size=bs)

NameError: name 'batch_size' is not defined